In [ ]:
import findspark
findspark.init("/opt/manual/spark/")

from pyspark.sql import SparkSession, functions as F

In [2]:
spark = SparkSession.builder \
.appName("ML Homework") \
.master("local[2]") \
.getOrCreate()

2022-02-12 20:41:14,367 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-12 20:41:15,803 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.format("csv") \
.option("header",True) \
.option("inferSchema",True) \
.option("sep",",") \
.load("file:///home/train/datasets/nyc_taxi.csv")

In [4]:
df1 = df.withColumn("pickup_datetime", F.to_timestamp(F.col("pickup_datetime"),"yyyy-MM-dd HH:mm:ss")) \
        .withColumn("dropoff_datetime", F.to_timestamp(F.col("dropoff_datetime"),"yyyy-MM-dd HH:mm:ss"))

In [5]:
df2 = df1 \
       .withColumn("pickup_year",
                    F.year(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_month",
                    F.month(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_dayofweek",
                    F.dayofweek(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_hour",
                    F.hour(F.col("pickup_datetime")))

In [6]:
def switch_tr_day(day_index):
    my_dict = {
        1: 'Pazar',
        2: 'Pazartesi',
        3: 'Salı',
        4: 'Çarşamba',
        5: 'Perşembe',
        6: 'Cuma',
        7: 'Cumartesi'
    }
    
    return my_dict.get(day_index)

def switch_month_day(month_index):
    my_dict = {
        1: 'Ocak',
        2: 'Subat',
        3: 'Mart',
        4: 'Nisan',
        5: 'Mayis',
        6: 'Haziran',
        7: 'Temmuz',
        8: 'Agustos',
        9: 'Eylul',
        10: 'Ekim',
        11: 'Kasim',
        12: 'Aralik'
    }
    
    return my_dict.get(month_index)

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r


In [7]:
from pyspark.sql.types import StringType, FloatType
from math import radians, cos, sin, asin, sqrt

haversine_distance = F.udf(lambda lon1, lat1, lon2, lat2: haversine(lon1, lat1, lon2, lat2), FloatType())
spark.udf.register("haversine_distance", haversine_distance)

switch_month = F.udf(lambda z: switch_month_day(z), StringType())
spark.udf.register("switch_month", switch_month)

switch_day_func = F.udf(lambda z: switch_tr_day(z), StringType())
spark.udf.register("switch_day_func", switch_day_func)

<function __main__.<lambda>(z)>

In [8]:
df3 = df2.withColumn("id", F.regexp_replace(F.col("id"), "[id]", "")) \
.withColumn("haversine_distance_km", haversine_distance(F.col("pickup_longitude"), F.col("pickup_latitude"), F.col("dropoff_longitude"), F.col("dropoff_latitude"))) \
.drop("store_and_fwd_flag", "pickup_year", "dropoff_datetime", "pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "id")

df3.limit(5).toPandas()

Traceback (most recent call last):                                  (0 + 1) / 1]
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


,vendor_id,passenger_count,trip_duration,pickup_month,pickup_dayofweek,pickup_hour,haversine_distance_km
0,2,1,455,3,2,17,1.498521
1,1,1,663,6,1,0,1.805507
2,2,1,2124,1,3,11,6.385098
3,2,1,429,4,4,19,1.485498
4,2,1,435,3,7,13,1.188589


In [9]:
df4 = df3.withColumnRenamed("trip_duration", "label")
df4.limit(10).toPandas()

Traceback (most recent call last):
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


,vendor_id,passenger_count,label,pickup_month,pickup_dayofweek,pickup_hour,haversine_distance_km
0,2,1,455,3,2,17,1.498521
1,1,1,663,6,1,0,1.805507
2,2,1,2124,1,3,11,6.385098
3,2,1,429,4,4,19,1.485498
4,2,1,435,3,7,13,1.188589
5,2,6,443,1,7,22,1.098943
6,1,4,341,6,6,22,1.326279
7,2,1,1551,5,7,7,5.714981
8,1,1,255,5,6,23,1.310353
9,2,1,1225,3,5,21,5.121161


In [ ]:
df4.printSchema()

In [11]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['vendor_id', 'passenger_count', 'pickup_month', 'pickup_dayofweek',
            'pickup_hour', 'haversine_distance_km'], outputCol='assembled_features', 
            handleInvalid='skip')

df5 = assembler.setHandleInvalid("skip").transform(df4)

## Scaler

In [12]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='assembled_features',
    outputCol='features')

scaler_model = scaler.fit(df5)

In [14]:
df6 = scaler_model.transform(df5)

df6.limit(5).toPandas()

,vendor_id,passenger_count,label,pickup_month,pickup_dayofweek,pickup_hour,haversine_distance_km,assembled_features,features
0,2,1,455,3,2,17,1.498521,"[2.0, 1.0, 3.0, 2.0, 17.0, 1.4985207319259644]","[4.009806753040021, 0.7608947760794935, 1.7846..."
1,1,1,663,6,1,0,1.805507,"[1.0, 1.0, 6.0, 1.0, 0.0, 1.8055071830749512]","[2.0049033765200104, 0.7608947760794935, 3.569..."
2,2,1,2124,1,3,11,6.385098,"[2.0, 1.0, 1.0, 3.0, 11.0, 6.385098457336426]","[4.009806753040021, 0.7608947760794935, 0.5948..."
3,2,1,429,4,4,19,1.485498,"[2.0, 1.0, 4.0, 4.0, 19.0, 1.4854984283447266]","[4.009806753040021, 0.7608947760794935, 2.3794..."
4,2,1,435,3,7,13,1.188589,"[2.0, 1.0, 3.0, 7.0, 13.0, 1.1885885000228882]","[4.009806753040021, 0.7608947760794935, 1.7846..."


In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression, RandomForestRegressor
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.functions import date_format, sin, cos, radians, atan2, month
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

 # Model
dtr = RandomForestRegressor(featuresCol="features", labelCol="label", impurity="variance")

# choices of tuning parameters
dtrparamGrid = (ParamGridBuilder()
    .addGrid(dtr.numTrees, [10,20,50])
    .addGrid(dtr.seed, [42])
    .addGrid(dtr.maxDepth, [5,10]).build())

# Pipeline

In [23]:
from pyspark.ml.pipeline import Pipeline

pipeline_obj = Pipeline(stages=[assembler, scaler, dtr])

In [24]:
pipeline_model = pipeline_obj.fit(df4)

Traceback (most recent call last):                                              
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [26]:
pipeline_model.write().overwrite().save("file:///home/train/project/nyc_taxi_model")

In [28]:
from pyspark.ml.pipeline import PipelineModel
loaded_pipeline_model = PipelineModel.load("file:///home/train/project/nyc_taxi_model")

In [35]:
pipeline_obj_nolabel = Pipeline(stages=[assembler, scaler])

pipeline_obj_nolabel_model = pipeline_obj_nolabel.fit(df4)

In [36]:
pipeline_obj_nolabel_model.write().overwrite().save("file:///home/train/project/nyc_taxi_model_nolabel_model")

# Cross Validation Model

In [50]:
pipeline = Pipeline(stages = [assembler, dtr])

crossval = CrossValidator(estimator = pipeline, estimatorParamMaps = dtrparamGrid, evaluator = RegressionEvaluator(labelCol = "label", predictionCol = "prediction", metricName = "rmse"), numFolds = 10)
model = crossval.fit(df4)

2022-02-09 23:30:45,184 WARN execution.CacheManager: Asked to cache already cached data.
2022-02-09 23:30:45,223 WARN execution.CacheManager: Asked to cache already cached data.


In [43]:
predictions = model.transform(df4).cache()
predictions.show(25)

+---------+---------------+-----+------------+----------------+-----------+---------------------+--------------------+------------------+
|vendor_id|passenger_count|label|pickup_month|pickup_dayofweek|pickup_hour|haversine_distance_km|            features|        prediction|
+---------+---------------+-----+------------+----------------+-----------+---------------------+--------------------+------------------+
|        2|              1|  455|           3|               2|         17|            1.4985207|[2.0,1.0,3.0,2.0,...| 780.4021992993382|
|        1|              1|  663|           6|               1|          0|            1.8055072|[1.0,1.0,6.0,1.0,...| 551.2517193947731|
|        2|              1| 2124|           1|               3|         11|            6.3850985|[2.0,1.0,1.0,3.0,...|1628.3537974683545|
|        2|              1|  429|           4|               4|         19|            1.4854984|[2.0,1.0,4.0,4.0,...| 780.4021992993382|
|        2|              1|  435| 

In [44]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol = "prediction", metricName = "rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 5121


In [45]:
evaluator2 = RegressionEvaluator(labelCol="label", predictionCol = "prediction", metricName = "mae")
mae = evaluator2.evaluate(predictions)
print("Mean Absolute Error (MAE) on train data = %g" % mae)

Mean Absolute Error (MAE) on test data = 422.498


In [29]:
test = spark.read.format("csv") \
.option("header",True) \
.option("inferSchema",True) \
.option("sep",",") \
.load("file:///home/train/datasets/test.csv")

In [30]:
test1 = test.withColumn("pickup_datetime", F.to_timestamp(F.col("pickup_datetime"),"yyyy-MM-dd HH:mm:ss"))

test2 = test1 \
       .withColumn("pickup_year",
                    F.year(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_month",
                    F.month(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_dayofweek",
                    F.dayofweek(F.to_date(F.col("pickup_datetime")))) \
       .withColumn("pickup_hour",
                    F.hour(F.col("pickup_datetime")))

testdf = test2.withColumn("id", F.regexp_replace(F.col("id"), "[id]", "")) \
.withColumn("haversine_distance_km", haversine_distance(F.col("pickup_longitude"), F.col("pickup_latitude"), F.col("dropoff_longitude"), F.col("dropoff_latitude"))) \
.drop("store_and_fwd_flag", "pickup_year", "pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "id")


In [32]:
transformed_df = loaded_pipeline_model.transform(testdf)
transformed_df.show()

+---------+---------------+------------+----------------+-----------+---------------------+--------------------+--------------------+------------------+
|vendor_id|passenger_count|pickup_month|pickup_dayofweek|pickup_hour|haversine_distance_km|  assembled_features|            features|        prediction|
+---------+---------------+------------+----------------+-----------+---------------------+--------------------+--------------------+------------------+
|        1|              1|           6|               5|         23|            2.7464259|[1.0,1.0,6.0,5.0,...|[2.00490337652001...| 849.5595431725029|
|        1|              1|           6|               5|         23|             2.759239|[1.0,1.0,6.0,5.0,...|[2.00490337652001...| 849.5595431725029|
|        1|              1|           6|               5|         23|            1.3061554|[1.0,1.0,6.0,5.0,...|[2.00490337652001...| 653.1291355384299|
|        2|              1|           6|               5|         23|             

Traceback (most recent call last):
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/manual/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [39]:
predictions = model.transform(testdf).cache()
predictions.show(25)

+---------+---------------+------------+----------------+-----------+---------------------+--------------------+------------------+
|vendor_id|passenger_count|pickup_month|pickup_dayofweek|pickup_hour|haversine_distance_km|            features|        prediction|
+---------+---------------+------------+----------------+-----------+---------------------+--------------------+------------------+
|        1|              1|           6|               5|         23|            2.7464259|[1.0,1.0,6.0,5.0,...| 773.8750799403113|
|        1|              1|           6|               5|         23|             2.759239|[1.0,1.0,6.0,5.0,...| 773.8750799403113|
|        1|              1|           6|               5|         23|            1.3061554|[1.0,1.0,6.0,5.0,...| 465.0740740740741|
|        2|              1|           6|               5|         23|             5.269088|[2.0,1.0,6.0,5.0,...| 1500.750883808626|
|        1|              1|           6|               5|         23|       

In [ ]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol = "prediction", metricName = "rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
evaluator2 = RegressionEvaluator(labelCol="label", predictionCol = "prediction", metricName = "mae")
mae = evaluator2.evaluate(predictions)
print("Mean Absolute Error (MAE) on test data = %g" % mae)

In [ ]:
model.write().overwrite().save("file:///home/train/project/nyc_taxi_model")

In [26]:
# Load the model

from pyspark.ml.tuning import TrainValidationSplitModel

loaded_pipeline_model = TrainValidationSplitModel.load("file:///home/train/project/nyc_taxi_model")

In [27]:
final_df = loaded_pipeline_model.transform(testdf)

In [28]:
final_df= final_df.drop("assembled_features","features")

In [29]:
type(final_df)

pyspark.sql.dataframe.DataFrame

# postgresql

In [30]:
jdbcUrl = "jdbc:postgresql://localhost/traindb?user=train&password=Ankara06"

In [31]:
final_df.write \
.mode("overwrite") \
.jdbc(url=jdbcUrl,
              table="final_df", 
              mode="overwrite", 
              properties={"driver": 'org.postgresql.Driver'})

`psql -U train -d traindb`

`select * from final_df limit 5;`

In [32]:
spark.stop()